![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Análisis de sentimientos y técnicas de NLP

En este taller podrán poner en práctica sus conocimientos sobre las diferentes técnicas para el procesamiento de lenguaje natural. El taller está constituido por 5 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción sentimientos de viajeros en Twitter

En este taller se usará el conjunto de datos de sentimientos sobre distintas aerolíneas de EE.UU. provenientes de Twitter. Cada observación contiene si el sentimiento de los tweets es positivo, neutral o negativo teniendo en cuenta distintas variables como aerolínea y las razones de los sentimientos negativos (como "retraso en el vuelo" o "servicio grosero"). El objetivo es predecir el sentimiento asociado a cada tweet. Para más detalles pueden visitar el siguiente enlace: [datos](https://www.kaggle.com/crowdflower/twitter-airline-sentiment).

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación de librerías
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# Lectura de la información de archivo .zip
tweets = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/Tweets.zip', index_col=0)

# Visualización dataset
tweets.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
tweet_id,,,,,,,,,,,,,,
570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
# Impresión tamaño del cojunto de datos
tweets.shape

(14640, 14)

### Análisis descriptivo

In [5]:
# Cuenta de tweets por cada sentimiento
tweets['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [6]:
# Cuenta de tweets por cada aerolínea
tweets['airline'].value_counts()

United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: airline, dtype: int64

In [7]:
# Plot con cuenta de tweets por cada aerolínea y sentimiento
pd.crosstab(index = tweets["airline"],columns = tweets["airline_sentiment"]).plot(kind='bar',figsize=(10, 6),alpha=0.5,rot=0,stacked=True,title="Sentiminetos por aerolínea")

### Liberias y Variables de interés y predicción

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Separación de variables predictoras (X) y de variable de interés (y)
X = tweets['text']
y = tweets['airline_sentiment'].map({'negative':-1,'neutral':0,'positive':1})

In [10]:
# Separación de datos en set de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Uso de CountVectorizer

En la celda 1 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función **CountVectorizer** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [11]:
# Celda 1
import re

# Función que realiza el preprocesamiento de texto

def preprocess_text(text):
    
    text = text.lower()                    # Se convierte el texto a minúsculas
    text = re.sub(r'\d+', '', text)        # Se eliminan los caracteres numéricos
    return text

# Se crea el vectorizador

vectorizer = CountVectorizer(lowercase=False, preprocessor=preprocess_text)

# Se crean los vectores a partir del texto y el vectorizador

X_traincv = vectorizer.fit_transform(X_train).toarray()

# Se utiliza un arbol de clasificación para entrenar el modelo y realizar las predicciones

clf = RandomForestClassifier()
clf.fit(X_traincv, y_train)
predictions=clf.predict(vectorizer.transform(X_test).toarray())

# Se reporta el accuracy del modelo

from sklearn.metrics import accuracy_score
acc_1=accuracy_score(y_test, predictions)
print(acc_1)

0.7702814569536424


### Punto 2 - Eliminación de Stopwords

En la celda 2 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función CountVectorizer, **eliminen stopwords** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [12]:
# Celda 2
# Se crea el vectorizador

vectorizer_2 = CountVectorizer(lowercase=False, preprocessor=preprocess_text, stop_words='english')
print(vectorizer_2.get_stop_words())

# Se crean los vectores a partir del texto y el vectorizador

X_traincv_2 = vectorizer_2.fit_transform(X_train).toarray()

# Se utiliza un arbol de clasificación para entrenar el modelo y realizar las predicciones

clf.fit(X_traincv_2, y_train)
predictions_2=clf.predict(vectorizer_2.transform(X_test).toarray())

# Se reporta el accuracy del modelo

from sklearn.metrics import accuracy_score
acc_2=accuracy_score(y_test, predictions_2)
print(acc_2)

frozenset({'who', 'their', 'under', 'along', 'she', 'less', 'nine', 'whither', 'same', 'you', 'un', 'latterly', 'top', 'bottom', 'beforehand', 'against', 'found', 'also', 'whereas', 'enough', 'few', 'are', 'several', 'someone', 'why', 'always', 'between', 'side', 'twenty', 'bill', 'may', 'neither', 'nevertheless', 'mine', 'her', 'ten', 'hers', 'cant', 'behind', 'do', 'is', 'via', 'only', 'within', 'ie', 'below', 'nothing', 'four', 'well', 'thereby', 'from', 'no', 'must', 'they', 'everything', 'please', 'thru', 'itself', 'alone', 'both', 'over', 'this', 'eleven', 'whole', 'of', 'etc', 'so', 'everyone', 'rather', 'elsewhere', 'by', 'once', 'becomes', 'three', 'he', 'seeming', 'meanwhile', 'amount', 'fifteen', 'other', 'much', 'himself', 'down', 'thick', 'anyway', 'per', 'yet', 'off', 'anything', 'everywhere', 'whom', 'themselves', 'keep', 'not', 'take', 'own', 'thus', 'about', 'however', 'seemed', 'fifty', 'wherein', 'these', 'for', 'or', 'anyhow', 'through', 'then', 'nowhere', 'ours', '

### Punto 3 - Lematización con verbos

En la celda 3 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función CountVectorizer, **lematizen el texto con verbos** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [13]:
# Celda 3

vect = CountVectorizer()
vect.fit(X_train)

# Obtención e impresión de los lemas de cada palabra de la lista asumiendo que cada palabra es un sustantivo
words = list(vect.vocabulary_.keys())[:100]
print([wordnet_lemmatizer.lemmatize(word) for word in words])

# Obtención e impresión de los lemas de cada palabra de la lista asumiendo que cada palabra es un verbo
print([wordnet_lemmatizer.lemmatize(word,pos='v') for word in words])

# Definición de la función que tenga como parámetro texto y devuelva una lista de lemas
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

# Creación de matrices de documentos usando CountVectorizer, usando el parámetro 'split_into_lemmas'
vect_lemas = CountVectorizer(analyzer=split_into_lemmas)

# Se crean los vectores a partir del texto y el vectorizador

X_traincv_3 = vect_lemas.fit_transform(X_train).toarray()

# Se utiliza un arbol de clasificación para entrenar el modelo y realizar las predicciones

clf.fit(X_traincv_3, y_train)
predictions_3=clf.predict(vect_lemas.transform(X_test).toarray())

# Se reporta el accuracy del modelo

from sklearn.metrics import accuracy_score
acc_3=accuracy_score(y_test, predictions_3)
print(acc_3)

['jetblue', 'of', 'course', 'am', 'americanair', 'complt', 'incompetence', 'on', 'flt', '295', 'lav', 'delay', 'from', 'pln', 'that', 'lnded', 'last', 'nite', 'no', 'internet', 'and', 'poor', 'svc', 'not', 'what', 'expect', 'southwestair', 'flight', '4968', 'n8325d', 'departs', 'mco', 'enroute', 'to', 'no_airport', 'http', 'co', 'fd4snvkiem', 'usairways', 'doe', 'anyone', 'your', 'airline', 'know', 'how', 'do', 'their', 'job', 'united', 'want', 'plane', 'dad', 'my', '386', 'dallas', 'jacksonville', 'fl', 'ha', 'been', 'cancelled', 'flightled', 'one', 'notified', 'me', 'going', 'it', 'is', 'trying', 'make', 'but', 'probably', 'too', 'late', 'did', 'see', 'an', 'email', 'could', 'call', 'pittsburgh', 'international', 'always', 'look', 'forward', 'jb', 'rt', 'bostongarden', 'looking', 'welcoming', 'you', 'onboard', 'have', 'the', 'ability', 'switch', 'cannot', 'tell', 'where', 'bag', 'are']
['jetblue', 'of', 'course', 'be', 'americanair', 'complt', 'incompetence', 'on', 'flt', '295', 'lav

### Punto 4 - Multiples técnicas

En la celda 4 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función **CountVectorizer, eliminen stopwords, lematizen el texto con verbos** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [14]:
# Celda 4
# Creación de matrices de documentos usando CountVectorizer, usando el parámetro 'split_into_lemmas'
vect_lemas_2 = CountVectorizer(analyzer=split_into_lemmas, stop_words='english')

# Se crean los vectores a partir del texto y el vectorizador

X_traincv_4 = vect_lemas_2.fit_transform(X_train).toarray()

# Se utiliza un arbol de clasificación para entrenar el modelo y realizar las predicciones

clf.fit(X_traincv_4, y_train)
predictions_4=clf.predict(vect_lemas_2.transform(X_test).toarray())

# Se reporta el accuracy del modelo

from sklearn.metrics import accuracy_score
acc_4=accuracy_score(y_test, predictions_4)
print(acc_4)

0.757864238410596


### Punto 5 - Comparación y análisis de resultados

En la celda 5 comparen los resultados obtenidos de los diferentes modelos (random forest) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

Comparación por Accuracy de los 4 modelos


In [15]:
# Celda 5

import matplotlib.pyplot as plt

print('Comparación por Accuracy de los 4 modelos')

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_title("Comparación de modelos")
ejeX = ['Acc 1', 'Acc 2', 'Acc 3', 'Acc 4']
ejeY = [round(acc_1, 4), round(acc_2, 4), round(acc_3, 4), round(acc_4, 4)]
ax.bar(ejeX,ejeY)

def addlabels(x,y,plotP):
    for i in range(len(x)):
        plotP.text(i,y[i],y[i],ha='center')
        
addlabels(ejeX,ejeY,plt)

plt.show()